In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role
# from sagemaker.image_uris import retrieve
from sagemaker.s3 import *
import sys

if int(sagemaker.__version__.split('.')[0]) == 2:
    !{sys.executable} -m pip install sagemaker==1.72.0
    print("Installing previous SageMaker Version. Please restart the kernel")
else:
    print("Version is good")

role = get_execution_role()
sess = sagemaker.Session(default_bucket=None)
region = boto3.session.Session().region_name
print("Region = {}".format(region))
sm = boto3.Session().client('sagemaker')


     |████████████████████████████████| 297 kB 29.1 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker: filename=sagemaker-1.72.0-py2.py3-none-any.whl size=388327 sha256=b0cda5670f00e9e1366bceeca88cddb32b2f76e917b25139cb58b7320e7e65fd
  Stored in directory: /home/ec2-user/.cache/pip/wheels/c3/58/70/85faf4437568bfaa4c419937569ba1fe54d44c5db42406bbd7
Successfully built sagemaker
  Attempting uninstall: smdebug-rulesconfig
    Found existing installation: smdebug-rulesconfig 1.0.1
    Uninstalling smdebug-rulesconfig-1.0.1:
      Successfully uninstalled smdebug-rulesconfig-1.0.1
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.86.2
    Uninstalling sagemaker-2.86.2:
      Successfully uninstalled sagemaker-2.86.2
Installing previous SageMaker Version. Please restart the kernel
Region = eu-central-1


In [2]:

# see https://sagemaker.readthedocs.io/en/stable/api/utility/image_uris.html for which inputs to use
# see https://github.com/aws/deep-learning-containers/blob/master/available_images.md for registry paths with custom algorithms

training_image = "763104351884.dkr.ecr.eu-central-1.amazonaws.com/pytorch-training:1.11.0-gpu-py38-cu115-ubuntu20.04-e3"
# training_image = "763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.6.0-gpu-py36-cu101-ubuntu16.04"
print (training_image)

763104351884.dkr.ecr.eu-central-1.amazonaws.com/pytorch-training:1.11.0-gpu-py38-cu115-ubuntu20.04-e3


In [3]:
!pip install sagemaker-experiments 
from sagemaker.analytics import ExperimentAnalytics
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

  Using cached sagemaker_experiments-0.1.35-py3-none-any.whl (42 kB)


In [4]:
rawbucket= sess.default_bucket() # Alternatively you can use our custom bucket here. 
data_bucket = "treetracker-training-images"
prefix = 'sagemaker-modelmonitor' # use this prefix to store all files pertaining to this workshop.

dataprefix = prefix + '/data'
traindataprefix = prefix + '/train_data'
testdataprefix = prefix + '/test_data'
testdatanolabelprefix = prefix + '/test_data_no_label'
trainheaderprefix = prefix + '/train_headers'

s3_train_key = "freetown/"
s3_validation_key = ""
s3_train = 's3://{}/{}/'.format(data_bucket, s3_train_key)
s3_validation = 's3://{}/{}/'.format(data_bucket, s3_validation_key)
sagemaker_train = 's3://{}/{}/'.format(rawbucket, s3_train_key)
sagemaker_validation = 's3://{}/{}/'.format(rawbucket, s3_validation_key)


In [5]:
train_data_location = rawbucket + '/' + traindataprefix
test_data_location = rawbucket+'/'+testdataprefix
print("Training data location = {}".format(train_data_location))
print("Test data location = {}".format(test_data_location))

Training data location = sagemaker-eu-central-1-053061259712/sagemaker-modelmonitor/train_data
Test data location = sagemaker-eu-central-1-053061259712/sagemaker-modelmonitor/test_data


In [6]:

from sagemaker.processing import ProcessingInput, ProcessingOutput

pre_input = ProcessingInput(source=s3_train, destination=None, input_name="pre")
pre_output = ProcessingOutput(source=None, destination=sagemaker_train)



In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='mobilenet_v2.py',
                    role=role,
                    framework_version='1.4.0',
                    train_instance_count=2,
                    train_instance_type='ml.c4.xlarge',
                    hyperparameters={
                        'epochs': 6,
                        'backend': 'gloo'
                    })